## Key terms

## Structural Patterns

Structural patterns are design patterns that ease the design by identifying a
simple way to realize relationships between entities.

Structural patterns are concerned with how classes and objects are composed
to form larger structures.


### Adapter

The adapter pattern is a software design pattern (also known as wrapper, an
alternative naming shared with the decorator pattern) that allows the interface
of an existing class to be used from another interface. It is often used to make
existing classes work with others without modifying their source code.

### Facade

A structural design pattern that provides a simplified interface to a library, a
framework, or any other complex set of classes.

## Adapter

The adapter pattern is a structural pattern that allows objects with incompatible
interfaces to collaborate.

We come across adapters in our day to day life. For example, we have a 3 pin plug
and we want to use it in a 2 pin socket. We can use an adapter to convert the 3 pin
plug to a 2 pin plug.

So, we use an adapter to allow two incompatible interfaces to work together.
Similarly, in software development, we have two incompatible interfaces and we want
to use them together. We can use an adapter to convert one interface to another. For
instance, we have an API that returns a list of users. Now the request to this API
requires a JSON object. Some clients instead of sending a JSON object, want to
send an XML object.

Should we change the API to accept an XML object? Should we create a new API that
accepts an XML object? No, that would be redundant. This is where the adapter
pattern comes into play. We can create an adapter that converts the XML object to a
JSON object and then use the existing API.

You can create an adapter. This is a special object that converts the interface of one
object so that another object can understand it.

An adapter wraps one of the objects to hide the complexity of conversion happening
behind the scenes. The wrapped object isnʼt even aware of the adapter. For example,
you can wrap an object that operates in meters and kilometers with an adapter that
converts all of the data to imperial units such as feet and miles.

Adapters can not only convert data into various formats but can also help objects
with different interfaces collaborate. Hereʼs how it works:

- The adapter gets an interface, compatible with one of the existing objects.
- Using this interface, the existing object can safely call the adapterʼs methods.
- Upon receiving a call, the adapter passes the request to the second object, but
in a format and order that the second object expects.

### Problem

Let us take the example of payment processing.
As a part of our application we want to integrate with different payment gateways.
We first use the Stripe payment gateway. The stripe team provides us with a library
that we can use to integrate with their payment gateway.

In [ ]:
class StripeApi:
    def create_payment(self) -> Payment:
        # Create payment
        pass

    def check_status(self, payment_id) -> PaymentStatus:
        # Check payment status
        pass

################################################################
# We use the Stripe API to create a payment and check the status of the payment.
################################################################

def process_payment():
    stripe_api = StripeApi()
    payment: Payment = stripe_api.create_payment()
    status: PaymentStatus = stripe_api.check_status(payment.id)

Now we want to integrate with another payment gateway. We use the PayPal
payment gateway. The PayPal team provides us with a library that we can use to
integrate with their payment gateway.

In [ ]:
class PayPalApi:
    def make_payment(self) -> Payment:
        # Create payment
        pass


    def get_status(self, payment_id) -> PaymentStatus:
        # Check payment status
        pass
    

As you can see, the Stripe API and the PayPal API have different method names. The
Stripe API uses create_payment and check_status while the PayPal API uses
make_payment and get_status .

Should we change where we use the Stripe API to use the PayPal API? No, that would
be redundant. That would require us to change the code in multiple places. Apart
from the additional work, it would also increase the chances of introducing bugs.
Also, when we want to switch back to the Stripe API, we would have to change the
code again. Hence, our code is also violating SRP and OCP. We are also using
concrete classes instead of interfaces. This makes our code tightly coupled.

### Implementation

#### 1. **Incompatible classes**

You should have two classes that have incompatible
interfaces. For example, the Stripe API and the PayPal API.

In [ ]:
class StripeApi:
    def create_payment(self) -> Payment:
        # Create payment
        pass

    def check_status(self, payment_id) -> PaymentStatus:
        # Check payment status
        pass

class PayPalApi:
    def make_payment(self) -> Payment:
        # Create payment
        pass

    def get_status(self, payment_id) -> PaymentStatus:
        # Check payment status
        pass

#### 2. **Adapter interface**

Create an interface for the adapter that will be used to
convert the incompatible interfaces.

In [ ]:
from abc import ABC, abstractmethod
class PaymentProvider(ABC):
    @abstractmethod
    def make_payment(self) -> Payment:
        pass
    
    @abstractmethod
    def get_status(self, payment_id) -> PaymentStatus:
        pass

#### 3. **Concrete adapter classes**

Create a class that implements the target
interface. This is the class that the client code expects to work with. The adapter
will convert the interface of the existing class to this interface.

In [ ]:
class StripeAdapter(PaymentProvider):
    def make_payment(self) -> Payment:
        # Create payment
        pass

    def get_status(self, payment_id) -> PaymentStatus:
        # Check payment status
        pass

class PayPalAdapter(PaymentProvider):
    def make_payment(self) -> Payment:
        # Create payment
        pass

    def get_status(self, payment_id) -> PaymentStatus:
        # Check payment status
        pass

#### 4. **Transform request and delegate to original class**

In the adapter class,
transform the request to the format that the original class expects. Then, call the original class to perform the operation.

In [ ]:
class StripeAdapter(PaymentProvider):
    def __init__(self):
        self.stripe_api = StripeApi()

    def make_payment(self) -> Payment:
        return self.stripe_api.create_payment()
    
    def get_status(self, payment_id) -> PaymentStatus:
        status = self.stripe_api.check_status(payment_id)
        return convert_status(status)

#### 5. **Client Code**

The client code expects to work with the target interface. The
client code doesnʼt know that the adapter is converting the interface of the
original class.


In [ ]:
class PaymentProcessor:
    def __init__(self, payment_provider: PaymentProvider):
        self.payment_provider = payment_provider
        
    def process_payment(self):
        payment = self.payment_provider.make_payment()
        status: PaymentStatus = self.payment_provider.get_status(payment.id)

#### Advantages

- You can use adapters to reuse existing classes with incompatible interfaces.
- You can even modify the request and response of the original classes.

- Single Responsibility Principle. You can separate the interface or data
conversion code from the primary business logic of the program.
- Open/Closed Principle. You can introduce new types of adapters into the
program without breaking the existing client code, as long as they work with the
adapters through the target interface.